In [1]:
import pandas as pd
import numpy as np
import plotnine as pn
from siuba import filter, group_by, gather, _, summarize, arrange, select, inner_join, mutate, ungroup, count

def plot_fix(wspace=0.25): 
    return pn.theme_minimal() + pn.theme(plot_background=pn.element_rect(fill="#FEF6E0", color="#FEF6E0"),  \
        strip_text = pn.element_text(  weight="bold"), subplots_adjust={'wspace':wspace}, panel_grid=pn.element_line(color="antiquewhite")) 

pd.set_option('display.max_colwidth', None)
pd.set_option("display.precision", 3)
pd.set_option('display.max_rows', 60)

In [2]:
# ml imports
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.model_selection import GridSearchCV

In [3]:
relative_path = "../../data/skade_txt/NL_slowchat_raw_data.pickle"
df = pd.read_pickle(relative_path) >> filter(~_.Text.isna())

In [4]:
saken = "(?:denne)? ?(?:saken|skadesaken|skadesak|sak|forsikringssaken|skaden|kravet|saka)"
avsluttes = "(?:da|derfor)? ?(?:avsluttes|slettes|avslutas|avslutast|avsluttast|kanselleres|lukkes|slettes|trekkes)"
avsluttet = "(?:avsluttet|avslutta|lukket|lukka|kansellert|behandlet|ferdigbehandlet|slettet|sletta|trukket)"
avslutte = "(?:avslutte|kansellere|lukke|slette)"
avslutter = "(?:avslutter|kansellerer|lukker|sletter|trekker|trekkjer)"
avslutt = "(?:avslutt|kanseller|lukk|slett)"
jeg = "(?:jeg|vi|eg)"
min = "(?:min|våres|vår)"
dere = "(?:du|dere|gjensidige)"
forsikringen = "(?:forsikringen|forsikring)"
brukt = "(?:brukt|benyttet)"
bruker = "(?:bruker|benytter|benyttjer|benyttar)"
bruke = "(?:bruke|benytte)"
ikke = "(?:ikke|ikkje)"
noe = "(?:noe|noko)"

regexes = [
    f"koster under egenandelen",
    f"{saken} kan {avsluttes}",
    f"ønsker {jeg}? ?{ikke} (?:å)? ?{bruke} {forsikringen}",
    f"Beløpet er under egenandel",
    f"{forsikringen} blir {ikke} {brukt}",
    f"(?:Bare|berre) å {avslutte} {saken}",
    f"kostnad(?:en)? er under egenandel",
    f"Kostnaden[e] for å utbedre skaden er under egenandelen",
    f"Velger å {ikke} {bruke} (?:meg|oss) av {forsikringen}",
    f"velger å {ikke} {bruke} {forsikringen} {min} på {saken}",
    f"{jeg} tar kostnaden selv",
    f"(?:anser vi|vi anser) {saken} som {avsluttet}",
    f"{dere} kan {avslutte} {saken}",
    f"{jeg} ønsker å {avslutte} {saken}",
    f"Har tatt regningen selv",
    f"beløp ble lavere enn egenandel",
    f"for å {ikke} {bruke} {forsikringen}",
    f"uten bruk av {forsikringen}",
    f"bestemt (?:meg|oss) for å {ikke} {bruke} {forsikringen}",
    f"{jeg} {avslutter} {saken}",
    f"Kan {dere} {avslutte} {saken}",
    f"kan vel {saken} {avsluttes}",
    f"{jeg} kan {avslutte} {saken}",
    f"Da {avslutter} {jeg} {saken} her",
    f"{jeg} ser {saken} som {avsluttet}",
    f"fra {min} side er {saken} {avsluttet}",
    f"Da kan {saken} {avsluttes}",
    f"Anser {saken} som {avsluttet}",
    f"Da er {saken} {avsluttet} fra {min} side",
    f"{saken} er løst\\.",
    f"{saken} kan (?:bare)? ?{avsluttes}",
    f"{jeg} ber om (?:at)? ?(?:saken|skadesaken) {avsluttes}",
    f"(?:mindre|lavere) enn egenandelen",
    f"{dere} bare {avslutter} {saken}",
    f"{saken} (?:skal|kan)? ?{avsluttes}",
    f"{saken} {avsluttes}",
    f"{dere} kan {avslutte} {saken}",
    f"Blir {ikke} {noe} krav i {saken}",
    f"Kan {dere} {avslutte} {saken}",
    f"Denne skal {avsluttes}",
    f"kan {avsluttes}\\.",
    f"{jeg} ønsker {ikke} å {bruke} (?:oss|meg) av {forsikringen}",
    f"Da er det bare å avslutte {saken}",
    f"{dere} kan (?:altså)? ?{avslutte} {saken}.",
    f"{jeg} {avslutter} (?:gjerne)? ?{saken}",
    f"{jeg} kan {avslutte} saken",
    f"{jeg} {avslutter} {saken}",
    f"Anser (?:da)? ?{saken} som {avsluttet}",
    f"kan {avslutte} {saken}",
    f"{saken} anses (?:som)? ?{avsluttet}",
    f"{saken} kan (?:vel)? ?{avsluttes}",
    f"{dere} kan (?:bare)? ?{avslutte} {saken}",
    f"så {saken} blir {ikke} tatt på {forsikringen}",
    f"{bruker} {ikke} {forsikringen}",
    f"{jeg} ønsker {dere} {avslutter} {saken}",
    f"{saken} kan {dere} {avslutte}",
    f"denne kan {avsluttes}",
    f"{saken} er bare å {avslutte}",
    f"{dere} kan {avslutte} den {saken}",
    f"den kan {avsluttes} for (?:våres|min) del",
    f"{saken} anses som {avsluttet}",
    f"uten å {bruke} {forsikringen}",
    f"Så da kan {saken} bare {avsluttes}",
    f"ber om at den {avsluttes}",
    f"{jeg} {avslutter} derfor {saken}",
    "kommer under egenandelen",
    f"{jeg} tok kostnaden selv",
    f"{jeg} vil {avslutte} {saken}",
    f"velger derfor å trekke {saken}",
    f"vennligst {avslutte} sak",
    f"bare {avslutt} denne {saken}",
    f"{saken} kan dermed {avsluttes}",
    f"dere kan {avslutte} den",
    f"{jeg} velger å ikke {bruke} {forsikringen}"
]

regex_total = f"(?:{'|'.join(regexes)})"

In [7]:
#for i, reg in enumerate(regexes):
#    print(i, reg)
#    print("")

In [5]:
df_matches = (
    df >>
    filter(_.Text.str.contains(regex_total, case=False, na=False)) >>
    select(_.Text)
)
df_matches.shape

(603, 1)

In [21]:
df_matches.to_excel("../../data/skadesak_matces.xlsx")

pandas.core.frame.DataFrame